In [20]:
!pip install pyautogen
!pip install openai
!pip install python-dotenv
!pip install playwright
!pip install beautifulsoup4
!pip install nest-asyncio
!pip install ipywidgets


[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: python3 -m pip install --upgrade pip

[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: python3 -m pip install --upgrade pip

[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: python3 -m pip install --upgrade pip

[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: python3 -m pip install --upgrade pip

[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: python3 -m pip install --upgrade pip

[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: python3 -m pip install --upgrade pip
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.4/139.4 kB 3.6 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 214.4/214.4 kB 5.8 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 35.2 MB/s eta 0:00:00:00:01

[not

In [21]:
!playwright install

161.3 MiB [                    ] 0% 10.7s161.3 MiB [                    ] 0% 3.4s161.3 MiB [                    ] 1% 2.8s161.3 MiB [                    ] 1% 2.4s161.3 MiB [=                   ] 2% 2.6s161.3 MiB [=                   ] 3% 2.3s161.3 MiB [=                   ] 4% 2.3s161.3 MiB [=                   ] 5% 2.2s161.3 MiB [=                   ] 6% 2.1s161.3 MiB [=                   ] 7% 2.1s161.3 MiB [==                  ] 8% 2.1s161.3 MiB [==                  ] 9% 2.0s161.3 MiB [==                  ] 10% 2.0s161.3 MiB [==                  ] 10% 2.1s161.3 MiB [==                  ] 11% 2.1s161.3 MiB [==                  ] 11% 2.2s161.3 MiB [==                  ] 11% 2.3s161.3 MiB [===                 ] 12% 2.3s161.3 MiB [===                 ] 12% 2.4s161.3 MiB [===                 ] 12% 2.5s161.3 MiB [===                 ] 12% 2.6s161.3 MiB [===                 ] 13% 2.7s161.3 MiB [===                 ] 14% 2.7s161.3 MiB [===                 ] 15% 2.7s161.3 MiB [===             

In [22]:
os.environ['AZURE_OAI_KEY'] = "38b2b9b8286c4123b3bf51cc07111785"
os.environ["AZURE_OAI_ENDPOINT"]="https://ik-oai-west-us.openai.azure.com/"
os.environ["BRAVE_API_TOKEN"]="BSAhYTht4n6HAm0NKA_emPGQPamY9MJ"


In [23]:
from dotenv import load_dotenv
import os
from autogen import UserProxyAgent, AssistantAgent
from openai import AzureOpenAI
from typing import Annotated
import httpx
import json
import subprocess

load_dotenv(override=True)

llm_config = {
    "model": "gpt-4o",
    "api_type": "azure",
    "api_key": os.environ['AZURE_OAI_KEY'],
    "base_url": os.environ['AZURE_OAI_ENDPOINT'],
    "api_version": "2024-02-01"
}

client = AzureOpenAI(
    # This is the default and can be omitted
    api_key=os.environ.get("AZURE_OAI_KEY"),
    api_version="2024-02-01",
    azure_endpoint = os.getenv("AZURE_OAI_ENDPOINT")
)

In [24]:
lowel_prompt = """### Role
You are a research agent specialized in insurance underwriting. 

### Instruction
Your task is to act as a research agent specialized in providing detailed information for insurance underwriting purposes, focusing on cancer and other life-threatening diseases. 

### Requirements
You MUST cover the following aspects for each disease:
- Description
- Disease classification
- Prevalence
- Cause
- Signs and symptoms
- Diagnosis and Investigations
- Treatment
- Prognosis (include remission and recurrence) & Complications
- Risk classification (e.g., stage, ocular MG, borderline hyperlipidaemia; familial hyperlipidaemia)
- References from reputable sources (WHO, government institutions, NGOs, clinics)
- Mortality risk – Life expectancy
- Morbidity risk – Hospitalisation risk; CI risk
- Co-morbidity risk
- Life expectancy under different stages and scenarios
- Hospitalisation risk and rate
- Cancer risk

### Example
Provide the information in an easy-to-read markdown format, using bullet points and tables wherever possible.

### Context and Data 
Ensure that your answer is factual, unbiased, and avoids relying on stereotypes. Use only reputable sources for references.
You are encourage to use scrape_web to find as much detail as you can, as brave_search often give incomplete information.

### Sample Output Primer:
Here is the beginning of an example output for cancer to serve as a template:

```markdown
## Disease: Lung Cancer

### Description
- **Lung cancer** is a type of cancer that begins in the lungs. The lungs are two spongy organs in your chest that take in oxygen when you inhale and release carbon dioxide when you exhale.

### Disease Classification
- **Small cell lung cancer (SCLC)**
- **Non-small cell lung cancer (NSCLC)**

### Prevalence
- Global Incidence: 2.2 million new cases annually (Source: WHO)
- Higher prevalence in smokers and older adults

### Cause
- Primary cause: smoking
- Other factors: exposure to radon gas, secondhand smoke, asbestos, and other carcinogens

### Signs and Symptoms
- Persistent cough
- Coughing up blood
- Shortness of breath
- Chest pain
- Hoarseness
- Weight loss

### Diagnosis and Investigations
- Imaging tests: Chest X-ray, CT scan
- Tissue sampling (biopsy)
- Molecular testing for specific gene mutations

### Treatment
- Surgery
- Radiation therapy
- Chemotherapy
- Targeted drug therapy
- Immunotherapy

### Prognosis (include remission and recurrence) & Complications
- **Prognosis:**
  - Early-stage: Higher likelihood of remission
  - Advanced-stage: Lower survival rates
- **Recurrence:**
  - Common in advanced stages
- **Complications:**
  - Metastasis to bones, brain, liver
  - Respiratory failure

### Risk Classification
- **Stage**: Based on TNM system
  - T: Size of the primary tumor
  - N: Extent of spread to lymph nodes
  - M: Presence of metastasis

### Mortality Risk – Life Expectancy
- Early-stage: 5-year survival rate ~60%
- Advanced-stage: 5-year survival rate ~15%

### Morbidity Risk
- Hospitalization risk: High, especially post-diagnosis
- Critical illness risk: Significant during treatment

### Co-morbidity Risk
- Higher in patients with existing respiratory conditions

### Life Expectancy Under Different Stages and Scenarios
- Early detection and treatment significantly improve life expectancy.

### Hospitalization Risk and Rate
- Hospitalization rates peak during initial diagnosis and treatment phases.

### Cancer Risk
- Increased risk for individuals with a history of smoking.

### References
- WHO: [Link]
- National Cancer Institute: [Link]

---
<Additional diseases will follow a similar structure>
```"""

julius_prompt = """### Role
You are a research agent specialized in insurance underwriting. 

### Instruction
Your task is to research unfamiliar diseases upon request and provide concise, deeply detailed information from reputable sources. Ensure all information is accurate and properly cited. 
You are provided with tools to help complete your task. Search the web for relevant webpages, and when needed navigate the url for detailed information.

### Research Criteria:
1. **Introduction**: Provide a brief overview of the disease (approx. 100 words).
2. **Symptoms and Effects**: List the symptoms and explain how they affect the patient.
3. **Treatment and Costs**: Describe available treatment methods and their associated costs.
4. **Prognosis and Statistics**: Detail the prognosis of the disease, including survival and recovery statistics.
5. **Major Complications**: Highlight the major complications associated with the disease.
6. **Causes**: Explain the main causes of the disease.

### Response Requirements:
- Responses should be in bullet points for easy reading.
- Information should be deeply detailed.
- Cite the reputable and trustworthy sources used (Sources to be provided).
- Ensure that your answer is unbiased and avoids relying on stereotypes.

### Quality Control
- You are encourage to use scrape_web to ensure detailed and accurate information.

### Example:
**Disease Name: [Insert Disease Name]**

**Introduction:**
[Provide a brief overview of the disease (approx. 100 words).]

**Symptoms and Effects:**
- Symptom 1: Detailed description
- Symptom 2: Detailed description
- ...

**Treatment and Costs:**
- Treatment Method 1: Description and cost
- Treatment Method 2: Description and cost
- ...

**Prognosis and Statistics:**
- Prognosis details
- Survival rates: Specific statistics
- Recovery rates: Specific statistics

**Major Complications:**
- Complication 1: Detailed description
- Complication 2: Detailed description
- ...

**Causes:**
- Cause 1: Detailed description
- Cause 2: Detailed description
- ...

**Sources:**
- Source 1
- Source 2"""

In [ ]:
from IPython.display import display, Markdown

language_country_map = {
    'en':'ALL',
    'jp':'JP'
}

async def research(input_text:str, language:str='en'):
    
    user_proxy_agent = UserProxyAgent(
        name='user_proxy_agent',
        is_termination_msg=lambda msg: msg.get('content', None) != None and msg.get('tool_calls', None) == None,
        code_execution_config=False,
        human_input_mode="NEVER",
        llm_config=llm_config
    )

    # created by FWD_AI_STUDIO

    uw_researcher_prompt = lowel_prompt
    
    uw_researcher = AssistantAgent(
        name = 'uw_researcher',
        system_message=uw_researcher_prompt,
        llm_config = llm_config,
        human_input_mode="NEVER",
    )

    # register the search tool for the uw_research
    # alternative sources are highly encouraged

    @user_proxy_agent.register_for_execution()
    @uw_researcher.register_for_llm(description="Search the web with Brave API to get information. Search for one concept at a time.")
    async def brave_search(query:Annotated[str, 'The search query. Could be a keyword, a question or a sentence.']) -> str:
        response = httpx.get(
            'https://api.search.brave.com/res/v1/web/search',
            params={
                'q':query,
                # 'search_lang':language,
                # 'country':language_country_map[language]
            },
            headers={
                'X-Subscription-Token':os.environ.get('BRAVE_API_TOKEN')
            }
        )

        results = response.json().get('web').get('results')[:1]
        results = [
            {k:result.get(k, None) for k in ['title', 'description', 'extra_snippets', 'url', 'page_age']}
            for result in results
        ]

        for result in results:
            url = result.get('url')
            output = subprocess.run(
                f'python3 scrape_web.py {url}',
                shell=True,
                capture_output=True,
            )
            result.update(
                {
                    'page_content':output.stdout.decode("utf-8")
                }
            )

        return json.dumps(results)

    language_prompt = ''
    if language == 'en':
        language_prompt = ''
    elif language == 'jp':
        language_prompt = ' 最終報告書を日本語で書く'
    
    res = await user_proxy_agent.a_initiate_chat(
        recipient=uw_researcher,
        clear_history=True,
        message = f'{input_text}{language_prompt}'
    )
    
    display(Markdown(res.chat_history[-1].get('content')))


In [ ]:
research("heart attact", "en")

In [ ]:
from ipywidgets import widgets, Button, HBox
from asyncio import get_running_loop
import nest_asyncio
nest_asyncio.apply()

text = widgets.Text(
    value='',
    placeholder='an unfamiliar disease',
    disabled=False   
)

language = widgets.RadioButtons(
    options=['en', 'jp']
)

button = Button(
    description='🔍'
)

button.on_click(
    lambda x: get_running_loop().run_until_complete(research(text.value, language.value))
)

hbox = HBox(
    children=[
        text,
        button,
        language
    ]
)

hbox